# Time domain averages (wide)

Read and check the epochs

In [1]:
import pandas as pd
from spudtr import epf, DATA_DIR, P3_F

epochs_df = pd.read_hdf(DATA_DIR / P3_F, key="p3").query('stim in ["target", "standard"]')
eeg_channels = ['MiPf', 'MiCe', 'MiPa', 'MiOc']

epf.check_epochs(epochs_df, eeg_channels, epoch_id="epoch_id", time="time_ms")
epochs_df

,epoch_id,time_ms,event_code,eeg_artifact,participant,MiPf,MiCe,MiPa,MiOc,A2,stim,accuracy,acc_type,exp
0,0,-100,0,False,demonstration,-48.0,23.015625,46.031250,11.656250,9.843750,target,correct,hit,p3
1,0,-96,0,False,demonstration,-52.5,19.984375,41.968750,6.800781,5.660156,target,correct,hit,p3
2,0,-92,0,False,demonstration,-51.5,22.765625,43.187500,7.773438,10.093750,target,correct,hit,p3
3,0,-88,0,False,demonstration,-54.0,21.750000,38.875000,5.101562,5.906250,target,correct,hit,p3
4,0,-84,0,False,demonstration,-55.0,19.984375,34.812500,5.343750,7.628906,target,correct,hit,p3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204045,741,980,0,False,demonstration,14.0,-13.156250,-9.781250,-4.128906,-4.183594,standard,correct,cr,p3
204046,741,984,0,False,demonstration,24.0,-1.264648,1.431641,3.158203,2.953125,standard,correct,cr,p3
204047,741,988,0,False,demonstration,25.0,3.289062,8.351562,7.042969,5.906250,standard,correct,cr,p3
204048,741,992,0,False,demonstration,23.5,0.252930,9.539062,10.203125,4.921875,standard,correct,cr,p3


Group by `time` to compute the time-domain average of all epochs and select columns of interest

In [2]:
grand_wide = epochs_df.groupby(['time_ms']).mean()[eeg_channels]
grand_wide.columns.name = 'channel'
grand_wide

channel,MiPf,MiCe,MiPa,MiOc
time_ms,,,,
-100,-1.649493,0.013497,-0.215728,-0.243184
-96,-1.661318,-0.206032,-0.328609,-0.388177
-92,-1.929899,-0.228171,-0.272750,-0.424717
-88,-1.920608,0.007321,-0.128252,-0.291439
-84,-1.827703,0.339352,0.178070,-0.011707
...,...,...,...,...
980,1.857264,1.209360,0.626984,-0.227464
984,1.966216,1.081492,0.463192,-0.375939
988,1.782095,0.854098,0.279379,-0.597021


Group by `time` and other columns to compute the average of subsets of epochs

In [3]:
subsets_wide = epochs_df.groupby(["time_ms", "stim"]).mean()[eeg_channels]
subsets_wide.columns.name = "channel"
subsets_wide

channel               MiPf      MiCe      MiPa      MiOc
time_ms stim                                            
-100    standard -0.660959  0.817002 -0.908828 -0.961365
        target   -2.611667 -0.768581  0.458890  0.455845
-96     standard -0.508562  0.823664 -0.769280 -0.930881
        target   -2.783333 -1.208268  0.100312  0.140055
-92     standard -0.674658  0.857362 -0.784269 -1.030781
...                    ...       ...       ...       ...
 988    target    6.156667  1.245361  2.337099  1.148226
 992    standard -2.910959  0.338024 -1.784356 -2.559927
        target    5.865000  1.238512  2.224318  1.238397
 996    standard -3.248288  0.053507 -1.933761 -2.869484
        target    5.926667  1.364069  2.367246  1.387686

[550 rows x 4 columns]

# Time-domain averages (long)

In [4]:
subsets_long = subsets_wide.stack()  # pivot the channel columns into one long column
subsets_long.name = "microvolts"
pd.DataFrame(subsets_long)

microvolts
time_ms stim     channel            
-100    standard MiPf      -0.660959
                 MiCe       0.817002
                 MiPa      -0.908828
                 MiOc      -0.961365
        target   MiPf      -2.611667
...                              ...
 996    standard MiOc      -2.869484
        target   MiPf       5.926667
                 MiCe       1.364069
                 MiPa       2.367246
                 MiOc       1.387686

[2200 rows x 1 columns]

# Time interval measurments

Interval measurments use the "slice-groupby-apply" pattern.  

* slice the time interval rows

* group by epoch_id and other tags

* apply the measurment function to the data, e.g., pandas built-in or user-defined


Start by doing the steps separately to verify.

When the steps are right, chain them for compact expression.

Example: single trial mean amplitude

1. Load the epochs

In [15]:
import pandas as pd
from spudtr import epf, DATA_DIR, P3_F

eeg_channels = ["MiPf", "MiCe", "MiPa", "MiOc"]

epochs_df = pd.read_hdf(DATA_DIR / P3_F, key="p3").query('stim in ["target", "standard"]')
epf.check_epochs(epochs_df, eeg_channels, epoch_id="epoch_id", time="time_ms")
epochs_df

,epoch_id,time_ms,event_code,eeg_artifact,participant,MiPf,MiCe,MiPa,MiOc,A2,stim,accuracy,acc_type,exp
0,0,-500,0,0,demonstration,-51.0,29.343750,40.312500,7.531250,7.382812,target,correct,hit,p3
1,0,-496,0,0,demonstration,-47.5,32.375000,43.406250,11.414062,8.367188,target,correct,hit,p3
2,0,-492,0,0,demonstration,-49.0,29.843750,41.750000,7.773438,3.937500,target,correct,hit,p3
3,0,-488,0,0,demonstration,-50.5,28.828125,40.062500,7.773438,1.722656,target,correct,hit,p3
4,0,-484,0,0,demonstration,-50.0,30.093750,37.687500,3.644531,0.984375,target,correct,hit,p3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278245,741,980,0,0,demonstration,14.0,-13.156250,-9.781250,-4.128906,-4.183594,standard,correct,cr,p3
278246,741,984,0,0,demonstration,24.0,-1.264648,1.431641,3.158203,2.953125,standard,correct,cr,p3
278247,741,988,0,0,demonstration,25.0,3.289062,8.351562,7.042969,5.906250,standard,correct,cr,p3
278248,741,992,0,0,demonstration,23.5,0.252930,9.539062,10.203125,4.921875,standard,correct,cr,p3


2. (optional) select the data columns of interest or skip this and use all of them

In [16]:
coi = ["epoch_id", "time_ms", "stim", "MiPf", "MiCe", "MiPa", "MiOc"]
mid = epochs_df[coi]
display(mid.head())
display(mid.tail())

,epoch_id,time_ms,stim,MiPf,MiCe,MiPa,MiOc
0,0,-500,target,-51.0,29.343750,40.31250,7.531250
1,0,-496,target,-47.5,32.375000,43.40625,11.414062
2,0,-492,target,-49.0,29.843750,41.75000,7.773438
3,0,-488,target,-50.5,28.828125,40.06250,7.773438
4,0,-484,target,-50.0,30.093750,37.68750,3.644531


,epoch_id,time_ms,stim,MiPf,MiCe,MiPa,MiOc
278245,741,980,standard,14.0,-13.156250,-9.781250,-4.128906
278246,741,984,standard,24.0,-1.264648,1.431641,3.158203
278247,741,988,standard,25.0,3.289062,8.351562,7.042969
278248,741,992,standard,23.5,0.252930,9.539062,10.203125
278249,741,996,standard,14.5,-10.367188,3.339844,7.289062


3. Slice the time interval data sample (rows) to measure and verify by inspection

In [17]:
mid_300_500 = mid.query("time_ms >= 300 and time_ms <= 500")

display(mid_300_500.head())
display(mid_300_500.tail())
display(mid_300_500["time_ms"].min(), mid_300_500["time_ms"].max())

,epoch_id,time_ms,stim,MiPf,MiCe,MiPa,MiOc
200,0,300,target,-46.0,69.5625,77.5000,29.640625
201,0,304,target,-41.5,78.4375,82.7500,33.531250
202,0,308,target,-39.0,83.1875,84.4375,33.031250
203,0,312,target,-39.5,81.9375,82.3125,29.875000
204,0,316,target,-36.5,82.6875,83.2500,31.828125


,epoch_id,time_ms,stim,MiPf,MiCe,MiPa,MiOc
278121,741,484,standard,14.5,3.541016,5.726562,5.585938
278122,741,488,standard,13.0,-4.300781,-5.484375,-1.943359
278123,741,492,standard,8.5,-6.578125,-10.015625,-1.214844
278124,741,496,standard,6.5,-11.382812,-14.789062,-0.971680
278125,741,500,standard,-1.5,-21.250000,-21.937500,-2.429688


300

500

4. **Group by** epoch_id(= single trial) and other column labels to preserve and **apply** the built-in `mean()` function.

  **Note** the `time_ms` timestamps just another column of data and also averaged
  
  **Note** `pandas` has several built-in stat functions besides mean: `max()`, `min()`, `var()`, `sd()`
 

In [18]:
mid_300_500_mna = mid_300_500.groupby(["epoch_id", "stim"]).mean()

display(mid_300_500_mna.head(), mid_300_500_mna.tail())

,,time_ms,MiPf,MiCe,MiPa,MiOc
epoch_id,stim,,,,,
0,target,400,-42.176472,42.853859,52.521751,12.860375
1,target,400,-14.617647,41.024815,42.625919,6.120811
2,target,400,-7.186275,24.073071,31.395679,13.836741
3,target,400,-16.911764,20.560892,26.349571,16.461147
4,target,400,13.039216,27.078394,22.416552,5.758588


,,time_ms,MiPf,MiCe,MiPa,MiOc
epoch_id,stim,,,,,
737,standard,400,13.911765,20.714920,22.612095,4.010857
738,standard,400,24.696079,-4.259727,0.098336,-2.500594
739,standard,400,27.578432,11.336646,14.499387,6.349677
740,standard,400,36.323528,-0.446557,3.442656,-0.209578
741,standard,400,11.490196,-12.447074,-6.483092,-2.429051


5. These are new data, re-label them appropriately

In [19]:
# drop the no longer meaningful time_ms column
mid_300_500_mna = mid_300_500_mna.drop("time_ms", axis=1)

# describe the type of measurment and interval
mid_300_500_mna["measure"] = "mna"
mid_300_500_mna["interval"] = "300_500"

display(mid_300_500_mna.head(), mid_300_500_mna.tail())

,,MiPf,MiCe,MiPa,MiOc,measure,interval
epoch_id,stim,,,,,,
0,target,-42.176472,42.853859,52.521751,12.860375,mna,300_500
1,target,-14.617647,41.024815,42.625919,6.120811,mna,300_500
2,target,-7.186275,24.073071,31.395679,13.836741,mna,300_500
3,target,-16.911764,20.560892,26.349571,16.461147,mna,300_500
4,target,13.039216,27.078394,22.416552,5.758588,mna,300_500


,,MiPf,MiCe,MiPa,MiOc,measure,interval
epoch_id,stim,,,,,,
737,standard,13.911765,20.714920,22.612095,4.010857,mna,300_500
738,standard,24.696079,-4.259727,0.098336,-2.500594,mna,300_500
739,standard,27.578432,11.336646,14.499387,6.349677,mna,300_500
740,standard,36.323528,-0.446557,3.442656,-0.209578,mna,300_500
741,standard,11.490196,-12.447074,-6.483092,-2.429051,mna,300_500


6. (optional) Export the measurements data

In [20]:
mid_300_500_mna.reset_index().to_feather(DATA_DIR / "p3_mid_mna_300_500.feather")

7. As above simplified by chaining and verified identical results

In [38]:
coi = ["epoch_id", "time_ms", "stim", "MiPf", "MiCe", "MiPa", "MiOc"]
mid_300_500_mna_c  = (
    epochs_df[coi]
    .query("time_ms >= 300 and time_ms <= 500")
    .groupby(["epoch_id", "stim"])
    .mean()
    .drop("time_ms", axis=1)
)

# describe the type of measurment and interval
mid_300_500_mna_c["measure"] = "mna"
mid_300_500_mna_c["interval"] = "300_500"

assert all(mid_300_500_mna_c == mid_300_500_mna)